In [80]:
import torch 
import torch.nn.functional as F
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader

import numpy as np
from numpy.random import normal

from MNISTDataset import MNISTDataset
from sklearn import datasets

torch.manual_seed(17)
np.random.seed(17)

N = 1600
batch = 32

digits = datasets.load_digits()

g_train = MNISTDataset(digits.data[:1600], digits.target[:1600], N)
data_loader = DataLoader(g_train, batch_size =  32, shuffle=True)

g_test = MNISTDataset(digits.data[1600:], digits.target[1600:], 1765-N)
test_loader = DataLoader(g_test, batch_size =  32, shuffle=True)

# simple NN:
class NN(nn.Module):   
    def __init__(self, n1, n2, n3):
        super().__init__()
        self.lin1 = nn.Linear(n1, n2)
        self.act = nn.ReLU()        
        self.lin2 = nn.Linear(n2, n3)
        self.soft = nn.Softmax()
        
                
    def forward(self, x):
        out = self.lin1(x)
        out = self.act(out)
        out = self.lin2(out)
        out = self.soft(out)

        return out 

inp = 8
net = NN(inp*inp, 100, 10)

loss = nn.NLLLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001)

for e in range(1):
    it_ = iter(data_loader)
    for i in range(1) :
        x, y = next(it_)
        
        x = x.float().reshape(-1, inp*inp).requires_grad_()
        y = torch.argmax(y, dim=1)
        
        out = net(x)  
        
        # logSoftmax:
        logout = torch.log(out)
        logout.retain_grad()
        
        # NLLLoss:
        l = loss(logout, y)
        l.retain_grad()
        
        # up to here it is working so fucking fine.
        # now debug the shit out of your backprop dude!
        
        print(f"Epoch {e+1} Iteration {i} Loss {l}")
        
        optimizer.zero_grad()
        l.backward()
        optimizer.step()

Epoch 1 Iteration 0 Loss 3.915478229522705


/home/ben-ks/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [81]:
W1 = net.lin1.weight
W2 = net.lin2.weight

In [88]:
W1.grad

tensor([[ 0.0000,  0.0000,  0.0290,  ...,  0.1403,  0.0747,  0.0018],
        [ 0.0000,  0.0000, -0.0044,  ..., -0.0206,  0.0176,  0.0000],
        [ 0.0000,  0.0000,  0.0039,  ...,  0.0257,  0.0090,  0.0000],
        ...,
        [ 0.0000, -0.0058, -0.0252,  ...,  0.1467,  0.0451,  0.0033],
        [ 0.0000,  0.0312,  0.1842,  ...,  0.0575,  0.0076, -0.0032],
        [ 0.0000,  0.0000,  0.0043,  ...,  0.0000,  0.0000,  0.0000]])

In [70]:
logout[0][y[0]]

tensor(-1.2030, grad_fn=<SelectBackward>)

In [68]:
y[1], l.data

(tensor(7), tensor(3.9155))

In [72]:
1/32

0.03125

In [31]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, inp*inp).float()
        labels = torch.argmax(labels, dim=1)
        outputs = net(images)
        
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')

Accuracy of the network on the 10000 test images: 85.45454545454545 %


/home/ben-ks/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
